<a href="https://colab.research.google.com/github/leonardo-muniz/IMDB-Sentiment-Analysis/blob/master/imdb_sentiment_analysis_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets

  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2025.3.0-py3-none-any.whl (193 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.7.0
    Uninstalling fsspec-2025.7.0:
      Successfully uninstalled fsspec-2025.7.0


In [3]:
from datasets import load_dataset

In [4]:
def load_and_inspect_dataset():
  """Carrega o dataset IMDB e exibe informações básicas."""
  print("Iniciando o carregamento do dataset IMDB...")
  try:
    # Carrega o dataset do Hugging Face
    dataset = load_dataset("imdb")

    train_data = dataset["train"]
    test_data = dataset["test"]

    print(f"Reviews no conjunto de treino: {len(train_data)}")
    print(f"Reviews no conjunto de teste: {len(test_data)}")

    print("\nCarregamento e inspeção básicos concluídos. Dataset pronto para pré-processamento!")

    return train_data, test_data

  except Exception as e:
    print(f"\nOcorreu um erro ao carregar o dataset: {e}")
    return None, None

In [5]:
train_data, test_data = load_and_inspect_dataset()

Iniciando o carregamento do dataset IMDB...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Reviews no conjunto de treino: 25000
Reviews no conjunto de teste: 25000

Carregamento e inspeção básicos concluídos. Dataset pronto para pré-processamento!


In [6]:
from transformers import AutoTokenizer

# Carregar o tokenizer para um modelo popular de classificação de texto
# 'distilbert-base-uncased' é uma boa escolha para começar: leve e eficiente.
# 'uncased' significa que o texto será convertido para minúsculas automaticamente.
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

print("Tokenizer carregado com sucesso!")
print(f"Exemplo de como o tokenizer funciona para uma frase simples: {tokenizer('Isso é um teste de tokenização.')}")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Tokenizer carregado com sucesso!
Exemplo de como o tokenizer funciona para uma frase simples: {'input_ids': [101, 26354, 2080, 1041, 8529, 3231, 2063, 2139, 19204, 21335, 20808, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [7]:
def preprocess_function(examples):
    # O tokenizer lida com a tokenização, conversão para IDs, e adição de attention_mask.
    # truncation=True garante que as sequências mais longas que o limite do modelo sejam cortadas.
    # max_length pode ser especificado se você quiser um comprimento máximo diferente do padrão do modelo.
    return tokenizer(examples["text"], truncation=True)

# Aplicar a função de pré-processamento aos conjuntos de treino e teste
# batched=True processa os exemplos em lotes, o que é mais eficiente.
# remove_columns=['text'] remove a coluna de texto original, pois não precisaremos dela mais.
# Isso ajuda a economizar memória e focar nas colunas processadas.
tokenized_train_data = train_data.map(preprocess_function, batched=True, remove_columns=["text"])
tokenized_test_data = test_data.map(preprocess_function, batched=True, remove_columns=["text"])

print("\nDataset de treino tokenizado:")
print(tokenized_train_data)
print("\nDataset de teste tokenizado:")
print(tokenized_test_data)

# Você pode inspecionar um exemplo para ver as novas colunas
print("\nExemplo de dado tokenizado (primeiro item do treino):")
print(tokenized_train_data[0])

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]


Dataset de treino tokenizado:
Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 25000
})

Dataset de teste tokenizado:
Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 25000
})

Exemplo de dado tokenizado (primeiro item do treino):
{'label': 0, 'input_ids': [101, 1045, 12524, 1045, 2572, 8025, 1011, 3756, 2013, 2026, 2678, 3573, 2138, 1997, 2035, 1996, 6704, 2008, 5129, 2009, 2043, 2009, 2001, 2034, 2207, 1999, 3476, 1012, 1045, 2036, 2657, 2008, 2012, 2034, 2009, 2001, 8243, 2011, 1057, 1012, 1055, 1012, 8205, 2065, 2009, 2412, 2699, 2000, 4607, 2023, 2406, 1010, 3568, 2108, 1037, 5470, 1997, 3152, 2641, 1000, 6801, 1000, 1045, 2428, 2018, 2000, 2156, 2023, 2005, 2870, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 1996, 5436, 2003, 8857, 2105, 1037, 2402, 4467, 3689, 3076, 2315, 14229, 2040, 4122, 2000, 4553, 2673, 2016, 2064, 2055, 2166, 1012, 1999, 3327, 2016, 4122, 2000, 3579, 2014, 3086, 2015, 2000, 2437, 2070, 

In [8]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.4 MB/s eta 0:00:00


In [9]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
import numpy as np
import evaluate

# 1. Mapear os rótulos (labels)
# O dataset IMDB já tem labels 0 e 1. Para classificação, 0 e 1 são bons.
# Podemos renomear a coluna 'label' para 'labels' que é o padrão esperado pelo Trainer.
tokenized_train_data = tokenized_train_data.rename_column("label", "labels")
tokenized_test_data = tokenized_test_data.rename_column("label", "labels")

# Definir as colunas que serão usadas no treinamento
tokenized_train_data.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_test_data.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


# 2. Definir um DataCollator
# Este data collator irá dinamicamente padar os batches para o comprimento máximo em cada batch.
data_collator = DataCollatorWithPadding(tokenizer=tokenizer) # Corrected typo DataatorCollatorWithPadding to DataCollatorWithPadding
print("\nDataCollator com padding definido.")

# 3. Carregar o Modelo
# Vamos usar o mesmo modelo que o tokenizer (distilbert-base-uncased) para classificação de sequência.
# num_labels=2 porque temos duas classes: positivo e negativo.
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
print("\nModelo DistilBERT para classificação de sequência carregado.")

# 4. Definir métricas de avaliação
# Precisamos de uma função para calcular métricas durante o treinamento.
# Acurácia é uma boa métrica para classificação.
metric = evaluate.load("accuracy") # Corrected: use evaluate.load

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

print("Função de métricas de avaliação definida.")


DataCollator com padding definido.


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Modelo DistilBERT para classificação de sequência carregado.


Função de métricas de avaliação definida.


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
import numpy as np
import evaluate
import os # Necessário para verificar TPU_NAME
import torch # Necessário para torch.device

# Adicione estas importações e a configuração da TPU
import torch_xla.core.xla_model as xm
import torch_xla.debug.metrics as xm_metrics

def get_tpu_device():
    if 'TPU_NAME' not in os.environ:
        print("TPU não está disponível, verificando GPU...")
        if torch.cuda.is_available():
            print("GPU detectada.")
            return torch.device("cuda")
        else:
            print("Nenhuma GPU ou TPU detectada, usando CPU.")
            return torch.device("cpu")
    else:
        print("TPU detectada!")
        return xm.xla_device()

device = get_tpu_device()
print(f"Usando o dispositivo: {device}")

# ... (Carregamento do dataset e tokenizer)
# Ex: tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
# ... (Pré-processamento e tokenização)
# Ex: tokenized_train_data = ...
# ... (Renomear colunas e set_format)
# Ex: tokenized_train_data = tokenized_train_data.rename_column("label", "labels")

# 2. Definir um DataCollator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
print("\nDataCollator com padding definido.")

# 3. Carregar o Modelo
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
# Mova o modelo para a TPU/GPU/CPU detectada
model.to(device) # <--- ESSA LINHA É CRÍTICA PARA A TPU
print("\nModelo DistilBERT para classificação de sequência carregado e movido para o dispositivo.")

# 4. Definir métricas de avaliação
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)
print("Função de métricas de avaliação definida.")

# 5. Configurar os argumentos de treinamento
# Importante: Para TPUs, geralmente é melhor NÃO definir per_device_train_batch_size/eval_batch_size
# diretamente, pois o Trainer com XLA ajusta isso.
# Apenas aponte o Trainer para o dispositivo XLA.
# Entretanto, o Trainer da Hugging Face com PyTorch/XLA é um pouco mais complexo de configurar
# do que com GPUs. Para simplificar, o Trainer normalmente se encarrega de mover
# os dados para a TPU se 'device' for passado corretamente para o 'model'.

# O TrainingArguments não tem um argumento 'device'. O Trainer lida com isso.
# O ponto crucial é model.to(device)

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16, # Manter para o Trainer
    per_device_eval_batch_size=16,  # Manter para o Trainer
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=500,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
    # Não precisa adicionar arguments específicos para TPU aqui,
    # o Trainer da HF é inteligente o suficiente para usar o dispositivo do modelo.
)

# 6. Criar e iniciar o Trainer
trainer = Trainer(
    model=model, # Seu modelo JÁ DEVE ESTAR NO DISPOSITIVO (TPU) AQUI
    args=training_args,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_test_data,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("\nConfigurações do Trainer concluídas. Iniciando o treinamento...")

# Iniciar o treinamento
# trainer.train()

print("\nTreinamento concluído!")

print("\nAvaliação final no conjunto de teste:")
# results = trainer.evaluate()
print(results)

final_model_save_path = "./my_imdb_sentiment_model"
# trainer.save_model(final_model_save_path)
# tokenizer.save_pretrained(final_model_save_path)
print(f"\nModelo e Tokenizer salvos em: {final_model_save_path}")

TPU não está disponível, verificando GPU...
Nenhuma GPU ou TPU detectada, usando CPU.
Usando o dispositivo: cpu

DataCollator com padding definido.


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Modelo DistilBERT para classificação de sequência carregado e movido para o dispositivo.
Função de métricas de avaliação definida.


/tmp/ipython-input-10-3663553691.py:84: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



Configurações do Trainer concluídas. Iniciando o treinamento...

Treinamento concluído!

Avaliação final no conjunto de teste:
